<a href="https://colab.research.google.com/github/QColeman97/AudioTagger/blob/master/AudioTagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
!pip install sox

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns # for data visualization 

import IPython
import IPython.display as ipd #To play sound in notebook
import scipy as sci
import wave 
from pathlib import Path

from scipy.fftpack import fft #Fast Fourier Transformation 
from scipy.io import wavfile 

import librosa 

In [50]:
import os 
print(os.listdir("drive/My Drive/CSC490Final-AudioTagger"))

['FSDKaggle2018.audio_test', 'FSDKaggle2018.audio_train', 'test_post_competition_scoring_clips.csv', 'train_post_competition.csv']


In [51]:
INPUT_PATH ="drive/My Drive/CSC490Final-AudioTagger/"
audio_train_file = (INPUT_PATH + "FSDKaggle2018.audio_train")
audio_test_file = (INPUT_PATH + "FSDKaggle2018.audio_test")
train= pd.read_csv(INPUT_PATH + "train_post_competition.csv")


#scipy.wavfile.read returns rate of wave, and # of data read
filename = '/001ca53d.wav'
sample_rate, samples = wavfile.read(str(audio_train_file) + filename)
print(samples)
print(train.shape)

[-33 -32 -34 ...  -1  -1  -1]
(9473, 5)


In [52]:
print(train.head())

          fname         label  ...  freesound_id             license
0  00044347.wav        Hi-hat  ...         28739         Attribution
1  001ca53d.wav     Saxophone  ...        358827         Attribution
2  002d256b.wav       Trumpet  ...         10897  Creative Commons 0
3  0033e230.wav  Glockenspiel  ...        325017         Attribution
4  00353774.wav         Cello  ...        195688         Attribution

[5 rows x 5 columns]


# DATA PREPROCESSING 
Cut out silent parts 

Normalize wave form

In [53]:
df_train = pd.read_csv(INPUT_PATH +'/train_post_competition.csv')
df_test = pd.read_csv(INPUT_PATH + '/test_post_competition_scoring_clips.csv')
labels = df_train.label.unique()
label2int = {label:index for index, label in enumerate(labels)}
num_class = len(labels)
#Indices of manually verified training data
verifed_train = np.array(df_train[df_train.manually_verified == 1].index)
#array of labels in number form (0 = hi-hat, 1 = saxophone, etc)
plain_y_train = np.array([label2int[label] for label in df_train.label])

#np.set_printoptions(threshold=np.inf)
plain_y_train

array([ 0,  1,  2, ..., 12, 20, 17])

In [0]:
'''Two approaches:
  1) LH uses highest feature with only beginning sound. Useful info
     are usually in begging part of sample and support. 
     
  2) Splits sample and uses one that are long but with coarser feature. 
     Good for samples with contents in the middle or later part. Perfect 
     for ones that use entire wav file  
'''
confLH, confX = {}, {}
confs = [confLH, confX]
confLH['folder'] = Path('LH')
confX['folder'] = Path('X')

#configs for confLH: highest resolutions

confLH['sampling_rate'] = 44100
confLH['duration'] = 4
confLH['hop_length'] = 882 # 20ms
confLH['fmin'] = 20
confLH['fmax'] = confLH['sampling_rate'] // 2
confLH['n_mels'] = 128
confLH['n_fft'] = confLH['n_mels'] * 20
confLH['audio_split'] = 'head'
confLH['samples'] = confLH['sampling_rate'] * confLH['duration']
confLH['dims'] = (confLH['n_mels'], 1 + 
                  int(np.floor(confLH['samples']/confLH['hop_length'])), 1)


# Approach X uses longer sound, then it uses suppressed 
confX['sampling_rate'] = 26000
confX['duration'] = 6
confX['hop_length'] = 520 # 20ms
confX['fmin'] = 20
confX['fmax'] = confX['sampling_rate'] // 2
confX['n_mels'] = 48
confX['n_fft'] = confX['n_mels'] * 20
confX['audio_split'] = 'dont_crop'
confX['samples'] = confX['sampling_rate'] * confX['duration']
confX['dims'] = (confX['n_mels'], 1 + 
                  int(np.floor(confX['samples']/confX['hop_length'])), 1)




In [0]:
import librosa
import librosa.display

def read_audio(conf, pathname):
    #return audio time series and sampling rate 
    y, sr = librosa.load(pathname, sr=conf['sampling_rate'])
    # trim silence
    if 0 < len(y):
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf['samples']: # long enough
        if conf['audio_split'] == 'head':
            y = y[0:0+conf['samples']]
    else: # pad blank
        padding = conf['samples'] - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf['samples'] - len(y) - offset), 'constant')
    return y

def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf['sampling_rate'],
                                                 n_mels=conf['n_mels'],
                                                 hop_length=conf['hop_length'],
                                                 n_fft=conf['n_fft'],
                                                 fmin=conf['fmin'],
                                                 fmax=conf['fmax'])
    #convert spectrogram to decibel
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

def show_melspectrogram(mels, conf):
    librosa.display.specshow(mels, x_axis='time', y_axis='mel', 
                             sr=conf['sampling_rate'], hop_length=conf['hop_length'],
                            fmin=conf['fmin'], fmax=conf['fmax'])
    plt.colorbar(format='%+2.0f dB')
    plt.title('Log-frequency power spectrogram')
    plt.show()

def read_as_melspectrogram(conf, pathname, debug_display=False):
    x = read_audio(conf, pathname)
    mels = audio_to_melspectrogram(conf, x)
    if debug_display:
        IPython.display.display(IPython.display.Audio(x, rate=conf['sampling_rate']))
        show_melspectrogram(mels, conf)
    return mels

In [0]:
#spectograms are ndarray 
mels1 = read_as_melspectrogram(confLH, audio_train_file + '/' +
                       df_train.fname[0], debug_display=False)
mels_LH2 = read_as_melspectrogram(confLH, audio_train_file + '/' +
                                  df_train.fname[1], debug_display=False)

mels2 = read_as_melspectrogram(confX, audio_train_file + '/' + 
                       df_train.fname[0], debug_display=False)


In [57]:
print(mels1.size)
print(mels_LH2.size)
print(mels2.size)


25728
25728
27744


In [0]:
def split_long_data(conf, X):
    # Splits long mel-spectrogram data with small overlap
    L = X.shape[1]
    one_length = conf['dims'][1]
    loop_length = int(one_length * 0.9)
    min_length = int(one_length * 0.2)
    print(' sample length', L, 'to cut every', one_length)
    for idx in range(L // loop_length):
        cur = loop_length * idx
        rest = L - cur
        if one_length <= rest:
            yield X[:, cur:cur+one_length]
        elif min_length <= rest:
            cur = L - one_length
            yield X[:, cur:cur+one_length]

def convert_X(df, conf, datapath):
    # Convert all files listed on df.fname
    # Then generates X (contains mel-spectrograms)
    # and index mapping to original sample order
    X = []
    index_map = []
    for i, fname in enumerate(df.fname):
        print('processing', fname)
        data = read_as_melspectrogram(conf, datapath + '/' + fname)
        for chunk in split_long_data(conf, data):
            X.append(np.expand_dims(chunk, axis=-1))
            index_map.append(i)
    return np.array(X), np.array(index_map)

def convert_y_train(idx_train, plain_y_train):
    return np.array( [plain_y_train[idx] for idx in idx_train])

In [63]:
print('All samples will be cut per split length (=duration)')
X_train, idx_train = convert_X(df_train[:10], confX, audio_train_file)
y_train = convert_y_train(idx_train, plain_y_train)
print('Now original 10 samples were cut into ', len(idx_train), 'samples.')
print()
print('idx_train holds original sample index, y_train is also converted to have the same length with X_train/idx_train.')
print('idx_train', idx_train)
print('y_train', y_train)

All samples will be cut per split length (=duration)
processing 00044347.wav
 sample length 578 to cut every 301
processing 001ca53d.wav
 sample length 517 to cut every 301
processing 002d256b.wav
 sample length 301 to cut every 301
processing 0033e230.wav
 sample length 301 to cut every 301
processing 00353774.wav
 sample length 301 to cut every 301
processing 003b91e8.wav
 sample length 665 to cut every 301
processing 003da8e5.wav
 sample length 301 to cut every 301
processing 0048fd00.wav
 sample length 301 to cut every 301
processing 004ad66f.wav
 sample length 351 to cut every 301
processing 0063ab88.wav
 sample length 1483 to cut every 301
Now original 10 samples were cut into  16 samples.

idx_train holds original sample index, y_train is also converted to have the same length with X_train/idx_train.
idx_train [0 0 1 2 3 4 5 5 6 7 8 9 9 9 9 9]
y_train [0 0 1 2 3 4 4 4 5 6 7 8 8 8 8 8]


## **Dataset for training**

In [65]:
def datapath(conf, filename):
    return conf['folder'] / filename

def loaddata(conf, filename):
    return np.load(conf['folder'] / filename)

#### This is Toy example by default ####
TRYING_AS_TOY = False # False if you like creating full set

for conf in confs:
    conf['folder'].mkdir(parents=True, exist_ok=True)
    if TRYING_AS_TOY:
        for file in ['X_train', 'y_train', 'idx_train', 'X_test', 'idx_test']:
            shutil.copy(EXTRA/datapath(conf, file+'.npy'), datapath(conf, file+'.npy'))
        plain_y_train = np.load(EXTRA/'toy_plain_y_train.npy')
        train_verified_idx = np.load(EXTRA/'toy_train_verified_idx.npy')
        train_blacklist_index = np.load(EXTRA / 'toy_train_blacklist.npy')
    else:
        if not os.path.exists(datapath(conf, 'X_train.npy')):
            X_train, idx_train = convert_X(df_train, conf, audio_train_file)
            y_train = convert_y_train(idx_train, plain_y_train)
            np.save(datapath(conf, 'X_train.npy'), X_train)
            np.save(datapath(conf, 'y_train.npy'), y_train)
            np.save(datapath(conf, 'idx_train.npy'), idx_train)

            X_test, idx_test = convert_X(df_test, conf, audio_test_file)
            np.save(datapath(conf, 'X_test.npy'), X_test)
            np.save(datapath(conf, 'idx_test.npy'), idx_test)

processing 00044347.wav
 sample length 201 to cut every 201
processing 001ca53d.wav
 sample length 201 to cut every 201
processing 002d256b.wav
 sample length 201 to cut every 201
processing 0033e230.wav
 sample length 201 to cut every 201
processing 00353774.wav
 sample length 201 to cut every 201
processing 003b91e8.wav
 sample length 201 to cut every 201
processing 003da8e5.wav
 sample length 201 to cut every 201
processing 0048fd00.wav
 sample length 201 to cut every 201
processing 004ad66f.wav
 sample length 201 to cut every 201
processing 0063ab88.wav
 sample length 201 to cut every 201
processing 006f2f32.wav
 sample length 201 to cut every 201
processing 0075d39c.wav
 sample length 201 to cut every 201
processing 00780200.wav
 sample length 201 to cut every 201
processing 0079d310.wav
 sample length 201 to cut every 201
processing 0091fc7f.wav
 sample length 201 to cut every 201
processing 0097160c.wav
 sample length 201 to cut every 201
processing 00ad7068.wav
 sample length 2

KeyboardInterrupt: ignored